<a href="https://colab.research.google.com/github/chefinha-tech/acad_control/blob/main/verificador_certificado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import ssl
import socket
import datetime
import os

# --- Configurações de Alerta ---
DIAS_PARA_EXPIRAR_ALERTA = 30 # Notificar se faltarem X dias ou menos para o certificado expirar
ALERTA_LOG_FILE = "alerta_certificados.log" # Nome do arquivo de log onde os alertas serão gravados

def registrar_alerta_em_log(message):
    """
    Registra uma mensagem de alerta em um arquivo de log.
    """
    timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_entry = f"[{timestamp}] {message}\n"
    try:
        with open(ALERTA_LOG_FILE, "a") as f: # O 'a' significa append (adicionar ao final)
            f.write(log_entry)
        print(f"Alerta registrado em '{ALERTA_LOG_FILE}'")
    except Exception as e:
        print(f"Erro ao registrar alerta em arquivo: {e}")

def verificar_certificado_site(hostname, port=443):
    """
    Verifica a validade do certificado SSL/TLS de um site.
    Args:
        hostname (str): O nome do host (domínio) do site.
        port (int): A porta (padrão 443 para HTTPS).
    Returns:
        dict: Um dicionário com o status do certificado e informações.
    """
    context = ssl.create_default_context()
    try:
        with socket.create_connection((hostname, port)) as sock:
            with context.wrap_socket(sock, server_hostname=hostname) as ssock:
                cert = ssock.getpeercert()

                not_before_str = cert['notBefore']
                not_after_str = cert['notAfter']

                try:
                    def parse_cert_date(date_str):
                        if date_str.endswith(' GMT'):
                            dt_naive = datetime.datetime.strptime(date_str[:-4], '%b %d %H:%M:%S %Y')
                            return dt_naive.replace(tzinfo=datetime.timezone.utc)
                        else:
                            dt = datetime.datetime.strptime(date_str, '%b %d %H:%M:%S %Y %Z')
                            if dt.tzinfo is None:
                                dt = dt.replace(tzinfo=datetime.timezone.utc)
                            return dt

                    not_before = parse_cert_date(not_before_str)
                    not_after = parse_cert_date(not_after_str)

                except ValueError:
                    return {
                        "status": "Erro na data",
                        "hostname": hostname,
                        "detalhes": "Não foi possível parsear as datas de validade do certificado. Verifique o formato."
                    }

                current_time = datetime.datetime.now(datetime.timezone.utc)

                status = "Válido"
                dias_restantes = (not_after - current_time).days if current_time < not_after else None

                if current_time < not_before:
                    status = "Ainda não válido (data futura)"
                elif current_time > not_after:
                    status = "Ainda não válido (data futura)" # Corrigido para ser mais claro
                elif dias_restantes is not None and dias_restantes <= DIAS_PARA_EXPIRAR_ALERTA:
                    status = f"Válido, mas expira em {dias_restantes} dias"
                elif current_time > not_after: # Certificado expirou
                     status = "Expirado"


                return {
                    "status": status,
                    "hostname": hostname,
                    "not_before": not_before.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "not_after": not_after.strftime('%Y-%m-%d %H:%M:%S UTC'),
                    "dias_restantes": dias_restantes,
                    "issuer": dict(x[0] for x in cert['issuer'])
                }

    except ssl.SSLCertVerificationError as e:
        return {
            "status": "Erro de verificação SSL",
            "hostname": hostname,
            "detalhes": str(e)
        }
    except socket.gaierror:
        return {
            "status": "Erro de DNS",
            "hostname": hostname,
            "detalhes": "Não foi possível resolver o nome do host."
        }
    except ConnectionRefusedError:
        return {
            "status": "Conexão recusada",
            "hostname": hostname,
            "detalhes": "A conexão foi recusada pelo servidor. Porta incorreta ou firewall?"
        }
    except Exception as e:
        return {
            "status": "Erro inesperado",
            "hostname": hostname,
            "detalhes": str(e)
        }

# --- Como usar a automação ---
if __name__ == "__main__":
    sites_file = "sites.txt"
    sites_para_verificar = []

    if os.path.exists(sites_file):
        with open(sites_file, "r") as f:
            for line in f:
                site = line.strip()
                if site:
                    sites_para_verificar.append(site)
    else:
        print(f"Arquivo '{sites_file}' não encontrado. Criando um exemplo com sites padrão.")
        with open(sites_file, "w") as f:
            f.write("google.com\n")
            f.write("certifica.com.br\n")
            f.write("expired.badssl.com\n") # Para testar certificado expirado
            f.write("digicert.com\n") # Exemplo de site válido
        sites_para_verificar = ["google.com", "certifica.com.br", "expired.badssl.com", "digicert.com"]

    if not sites_para_verificar:
        print("Nenhum site encontrado para verificar.")
    else:
        print("Iniciando verificação de certificados SSL/TLS...\n")

        # Limpa o arquivo de log a cada nova execução, ou use "a" para manter o histórico
        # with open(ALERTA_LOG_FILE, "w") as f: # Use "w" para sobrescrever, "a" para adicionar
        #    f.write(f"--- Início da Verificação: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} ---\n")

        for site in sites_para_verificar:
            print(f"Verificando: {site}")
            resultado = verificar_certificado_site(site)
            for key, value in resultado.items():
                print(f"  {key}: {value}")
            print("-" * 30)

            # Lógica para registrar alertas no arquivo de log
            alerta_message = None
            if resultado["status"] == "Expirado":
                alerta_message = f"ALERTA CRÍTICO: O certificado para {resultado['hostname']} está EXPIRADO desde {resultado['not_after']}!"
            elif resultado["status"].startswith("Válido, mas expira"):
                alerta_message = f"AVISO: O certificado para {resultado['hostname']} expira em {resultado['dias_restantes']} dias!"
            elif resultado["status"].startswith("Erro"):
                alerta_message = f"ERRO na Verificação: Problema ao verificar o certificado de {resultado['hostname']}: {resultado['detalhes']}"

            if alerta_message:
                registrar_alerta_em_log(alerta_message)

        print(f"\nVerificação concluída. Verifique o arquivo '{ALERTA_LOG_FILE}' no painel de arquivos para ver os alertas.")

Iniciando verificação de certificados SSL/TLS...

Verificando: google.com
  status: Válido
  hostname: google.com
  not_before: 2025-06-02 08:35:30 UTC
  not_after: 2025-08-25 08:35:29 UTC
  dias_restantes: 60
  issuer: {'countryName': 'US', 'organizationName': 'Google Trust Services', 'commonName': 'WR2'}
------------------------------
Verificando: certifica.com.br
  status: Válido
  hostname: certifica.com.br
  not_before: 2025-02-07 00:00:00 UTC
  not_after: 2026-03-08 23:59:59 UTC
  dias_restantes: 256
  issuer: {'countryName': 'US', 'organizationName': 'DigiCert Inc', 'organizationalUnitName': 'www.digicert.com', 'commonName': 'RapidSSL TLS RSA CA G1'}
------------------------------
Verificando: globalsign.com
  status: Válido
  hostname: globalsign.com
  not_before: 2024-10-07 04:12:02 UTC
  not_after: 2025-11-06 14:31:01 UTC
  dias_restantes: 133
  issuer: {'countryName': 'BE', 'organizationName': 'GlobalSign nv-sa', 'commonName': 'GlobalSign Extended Validation CA - SHA256 - G3

In [21]:
!pip install cryptography


In [22]:
%%writefile sites.txt
google.com
certifica.com.br
globalsign.com
digicert.com
expired.badssl.com
sitequenaoexiste12345.com
mec.gov.br
emec.mec.gov.br

Overwriting sites.txt
